<a href="https://colab.research.google.com/github/Bfestus/medical_chatbot_qa/blob/main/medical_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ Step 1: Setup & Data Preparation

In [ ]:
# Basic Libraries
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/Medical_chatbot/medical_qa_doctor_style_refined.csv'
df = pd.read_csv(file_path)

# Clean dataset
df.drop_duplicates(inplace=True)
df['question'] = df['question'].str.strip()
df['answer'] = df['answer'].str.strip()
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Preview
print(f"Total cleaned samples: {len(df)}")
df = df.rename(columns={'question': 'input_text', 'answer': 'target_text'})
df['input_text'] = 'healthcare question: ' + df['input_text']


Mounted at /content/drive
Total cleaned samples: 285


✅ Step 2: Train/Val/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# 80-10-10 split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert to Hugging Face Datasets
from datasets import Dataset, DatasetDict
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(test_df)
})


✅ Step 3: Tokenization

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

max_input_length = 128
max_target_length = 128

def preprocess(example):
    inputs = tokenizer(example['input_text'], max_length=max_input_length, padding="max_length", truncation=True)
    targets = tokenizer(example['target_text'], max_length=max_target_length, padding="max_length", truncation=True)
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_datasets = dataset.map(preprocess, batched=True, remove_columns=dataset['train'].column_names)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/228 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

✅ Step 4: Model Setup

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, create_optimizer

# Load model
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

# Training parameters
batch_size = 8
epochs = 30
learning_rate = 5e-5
num_train_steps = (len(tokenized_datasets['train']) // batch_size) * epochs
optimizer, schedule = create_optimizer(init_lr=learning_rate, num_warmup_steps=0, num_train_steps=num_train_steps)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


✅ Step 5: TF Dataset Creation

In [ ]:
def create_tf_dataset_from_hf(dataset, data_collator, batch_size, shuffle=False):
    examples = [{
        "input_ids": example["input_ids"],
        "attention_mask": example["attention_mask"],
        "labels": example["labels"]
    } for example in dataset]

    def data_generator():
        indices = list(range(len(examples)))
        if shuffle:
            np.random.shuffle(indices)
        for i in range(0, len(indices), batch_size):
            batch = [examples[j] for j in indices[i:i + batch_size]]
            collated = data_collator(batch)
            yield (
                {"input_ids": np.array(collated["input_ids"]), "attention_mask": np.array(collated["attention_mask"])},
                np.array(collated["labels"])
            )

    return tf.data.Dataset.from_generator(
        data_generator,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(None, None), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(None, None), dtype=tf.int32)
            },
            tf.TensorSpec(shape=(None, None), dtype=tf.int32)
        )
    )

# Build TF datasets
tf_train_dataset = create_tf_dataset_from_hf(tokenized_datasets["train"], data_collator, batch_size, shuffle=True)
tf_val_dataset = create_tf_dataset_from_hf(tokenized_datasets["validation"], data_collator, batch_size, shuffle=False)


✅ Step 6: Model Training & Saving

In [ ]:
# Compile and train
model.compile(optimizer=optimizer)
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=epochs)

# Save model
output_dir = "/content/drive/MyDrive/Colab Notebooks/Medical_chatbot/healthcare-chatbot-model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")


Epoch 1/30
29/29 [==============================] - 89s 548ms/step - loss: 5.8964 - val_loss: 2.2403
Epoch 2/30
29/29 [==============================] - 4s 152ms/step - loss: 2.1928 - val_loss: 1.4714
Epoch 3/30
29/29 [==============================] - 5s 161ms/step - loss: 1.5327 - val_loss: 0.9013
Epoch 4/30
29/29 [==============================] - 4s 136ms/step - loss: 0.9885 - val_loss: 0.4240
Epoch 5/30
29/29 [==============================] - 4s 128ms/step - loss: 0.6011 - val_loss: 0.2382
Epoch 6/30
29/29 [==============================] - 4s 140ms/step - loss: 0.3976 - val_loss: 0.1600
Epoch 7/30
29/29 [==============================] - 4s 126ms/step - loss: 0.3083 - val_loss: 0.1145
Epoch 8/30
29/29 [==============================] - 3s 117ms/step - loss: 0.2499 - val_loss: 0.0908
Epoch 9/30
29/29 [==============================] - 3s 119ms/step - loss: 0.2132 - val_loss: 0.0756
Epoch 10/30
29/29 [==============================] - 3s 120ms/step - loss: 0.1986 - val_loss: 0.065

✅ Step 7: Inference Function

In [ ]:
# Keywords for filtering
medical_keywords = [
    "symptom", "diagnose", "treatment", "medicine", "disease", "doctor",
    "covid", "cancer", "diabetes", "bipolar", "stroke", "fever", "infection",
    "pain", "mental", "health", "hospital", "vaccine", "prescription",
    "disorder", "diagnosed", "asthma", "epilepsy", "hypertension",
    "depression", "anxiety", "hiv", "ibuprofen", "lisinopril", "side effects",
    "paracetamol", "atorvastatin", "metformin", "checkup", "healthy lifestyle",
    "symptoms", "water", "dose", "blood pressure", "heart", "immune",
    "medication", "mental health", "therapy"
]

def is_medical_question(question):
    return any(keyword in question.lower() for keyword in medical_keywords)

def generate_answer(question):
    if not is_medical_question(question):
        return "❗ Sorry, I can only answer healthcare-related questions."
    input_text = "healthcare question: " + question
    input_ids = tokenizer(input_text, return_tensors="tf", padding=True, truncation=True).input_ids
    output = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test examples
test_questions = [
    "What are the symptoms of stroke?",
    "Can bipolar disorder be detected early?",
    "How is COVID-19 diagnosed?",
]

for q in test_questions:
    print(f"\n❓ Question: {q}")
    print(f"💬 Answer: {generate_answer(q)}")



❓ Question: What are the symptoms of stroke?
💬 Answer: Symptoms of stroke include shortness of breath, fatigue. this information is helpful for understanding the condition better. understanding this response helps in gaining deeper insight into the medical condition and encourages timely medical consultation. If you have any concerns or symptoms, it's important to follow up with a healthcare provider for a personalized evaluation. If you have any concerns or symptoms, it's important to follow up with a healthcare provider for a personalized evaluation.

❓ Question: Can bipolar disorder be detected early?
💬 Answer: Yes, certainly, early detection of bipolar disorder is possible through questionnaires, ecg. this information is helpful for understanding the condition better. healthcare providers use behavioral assessments, interviews, and family history to evaluate and detect mood disorders at an early stage, potentially preventing more severe episodes. If you have any concerns or sympto